In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.text_data_utils.bert_embeddings import BertFeatureExtractor

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd
import joblib

In [4]:
!ls ../data/text_data/

'JDB_Face_metadata (1).json'   cleaned_DFDB_Face_metadata.csv


In [6]:
jdb_fp = '/home/ginger/code/gderiddershanghai/Diffusion-Face/data/text_data/JDB_Face_metadata (1).json'
df = pd.read_json(jdb_fp)

In [10]:
df.head()

,img_path,prompt,Task1,Task2,Task3,ori_prompt,url
0,84174671-387f-4927-9cd3-df194ac4bf79.jpg,"character art, agent of chaos, cinematic light...","{'Style': ['cinematic lighting', 'highly detai...",{'Caption': 'A highly detailed digital illustr...,{'Style Relevant Questions and Answers': [{'Qu...,"character art, agent of chaos, cinematic light...",https://cdn.discordapp.com/attachments/9954312...
1,2bb2dbb7-8e63-4cef-ac0a-1590cab19ed2.jpg,"Post Malone as Yoda, super realistic, star war...","{'Style': ['super realistic'], 'Content': ['Po...",{'Caption': 'Post Malone takes on the role of ...,{'Style Relevant Questions and Answers': [{'Qu...,"Post Malone as Yoda, super realistic, star war...",https://cdn.discordapp.com/attachments/9954312...
2,2864a9cb-82ce-4501-8306-c9f478533bf7.jpg,"Jim Lee style, James Tyler Williams Lincoln Pe...","{'Style': ['Jim Lee style', 'Harlem Renaissanc...",{'Caption': 'In this close-up image rendered i...,{'Style Relevant Questions and Answers': [{'Qu...,"Jim Lee style, James Tyler Williams Lincoln Pe...",https://cdn.discordapp.com/attachments/9954312...
3,513c2b14-6ba7-4cee-b3eb-5d64314c7ffe.jpg,weird Al yankovic as a poodle,"{'Style': ['weird', 'poodle'], 'Content': ['Al...",{'Caption': 'Weird Al Yankovic transformed int...,{'Style Relevant Questions and Answers': [{'Qu...,weird Al yankovic as a poodle,https://cdn.discordapp.com/attachments/9954315...
4,2e59debf-5727-4c9c-bc42-3e2b2559b8bb.jpg,hindu,"{'Style': ['Hindu art', 'Indian style', 'Relig...",{'Caption': 'A depiction of Hindu deities in a...,{'Style Relevant Questions and Answers': [{'Qu...,hindu --v 4,https://cdn.discordapp.com/attachments/9954315...


In [12]:
extractor = BertFeatureExtractor()
# new_embeddings = extractor.transform(df, text_column=text_column)

# #load model
# model = joblib.load(model_path)
# new_predictions_prob = model.predict_proba(new_embeddings)
# new_predictions = np.where(new_predictions_prob[:, 1] > threshold, 1, 0)

# df['nsfw_prediction'] = new_predictions
# df.to_csv(save_path, index=False)


/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to

In [13]:
new_embeddings = extractor.transform(df, text_column='prompt')

  0%|          | 0/61983 [00:00<?, ?it/s]

In [17]:
df['nsfw'] = np.nan
df['contains_person'] = np.nan

In [19]:
df = df[['img_path', 'prompt', 'nsfw', 'contains_person']]

In [21]:
model_path = '/home/ginger/code/gderiddershanghai/Diffusion-Face/weights/svm_model_weights_nsfw.pkl'
model = joblib.load(model_path)
new_predictions_prob = model.predict_proba(new_embeddings)


NameError: name 'threshold' is not defined

In [23]:
threshold = 0.85
new_predictions = np.where(new_predictions_prob[:, 1] > threshold, 1, 0)

In [24]:
save_path = '/home/ginger/code/gderiddershanghai/Diffusion-Face/data/text_data/JDB_Face.csv'
df['nsfw_prediction'] = new_predictions
df.to_csv(save_path, index=False)

In [25]:
df.head()

,img_path,prompt,nsfw,contains_person,nsfw_prediction
0,84174671-387f-4927-9cd3-df194ac4bf79.jpg,"character art, agent of chaos, cinematic light...",NaN,NaN,0
1,2bb2dbb7-8e63-4cef-ac0a-1590cab19ed2.jpg,"Post Malone as Yoda, super realistic, star war...",NaN,NaN,0
2,2864a9cb-82ce-4501-8306-c9f478533bf7.jpg,"Jim Lee style, James Tyler Williams Lincoln Pe...",NaN,NaN,0
3,513c2b14-6ba7-4cee-b3eb-5d64314c7ffe.jpg,weird Al yankovic as a poodle,NaN,NaN,0
4,2e59debf-5727-4c9c-bc42-3e2b2559b8bb.jpg,hindu,NaN,NaN,0
